# HeartLab Technical Test

This notebook outlines the process I underwent to create my CNN network which has approximately 90-92% validation accuracy.  
Note that as this is a distributed public Google Colab notebook, the files need to be downloaded and uploaded into the files workspace in order to function like it would in a local IDE environment. During production, the notebook was used in a private workspace which had much faster file import times compared to what you will experience.  
You may see this as substitute for the `main.py` script.  
For best results, please run the cells in sequential order.  
Please do reach out if there are any issues.  
Best regards,  
Jae Min 🌭

#Prelims
As we are running the code through Google Colab, we need to first upload files to allow this to work.
Please download the files from this public Google Drive folder: https://drive.google.com/drive/folders/1uI856or3wqje2YwJeOLq0AkaDUMvQ9Vy?usp=sharing  
You can upload them by dragging and dropping them onto the Files panel on the left (a runtime should be configured, and running on a (Google) GPU or TPU). Note that all files (`dataset.pickle`, `logs` (folder), `chest_xray` (folder), and `model.py`) should be unzipped, and found in the same directory as `'sample_data'`.

# Imports and Configs

In [ ]:
# Imports
import os
import time
import pickle

import cv2
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import TensorBoard

# Config and Inits
#data_dir = "drive/MyDrive/HeartLabTestFiles/chest_xray"
data_dir = "chest_xray"
size = (256, 256)
IMG_SIZE = 256

# Utils

In [ ]:
def img_2_arr(
    img_path: str,
    resize: bool = False,
    grayscale: bool = True,
    size: tuple = (256, 256),
) -> np.ndarray:

    """
    This function is responsible for opening an image, Preprocessing
    it by color or size and returning a numpy array.

    Input:
        - img_path: str, a path to the location of a image file on disk
        - resize: bool, True/False if the image is to be resized
        - grayscale: bool, True/False if image is meant to be B&W or color
        - size: tuple, a 2d tuple containing the x/y size of the image.

    Output:
        - a np.ndarray which is assosiated to the image that was input.
    """

    if grayscale:
        img_arr = cv2.imread(img_path, 0)
    else:
        img_arr = cv2.imread(img_path)

    if resize:
        img_arr = cv2.resize(img_arr, size)

    return img_arr


def create_datasets(data_dir: str) -> np.ndarray:
    """
    This function is responsible for creating a dataset which
    contains all images and their associated class.

    Inputs:
        - data_dir: str, which is the location where the chest x-rays are
            located.

    Outputs:
        - a np.ndarray which contains the processed image, and the class
            int, associated with that class.

    """
    # Image Loading and Preprocessing
    all_normal_img_paths = []
    all_viral_img_paths = []
    all_bact_img_paths = []
    for cls in os.listdir(data_dir): # NORMAL or PNEUMONIA
        for img in os.listdir(os.path.join(data_dir, cls)): # all images
            if cls == "NORMAL":
                all_normal_img_paths.append(os.path.join(data_dir, cls, img))
            elif "virus" in img:
                all_viral_img_paths.append(os.path.join(data_dir, cls, img))
            else:
                all_bact_img_paths.append(os.path.join(data_dir, cls, img))

    # 0 for normal, 1 for bacterial and 2 for viral
    dataset = (
        [
            [img_2_arr(path, grayscale=True, resize=True, size=size), 0]
            for path in all_normal_img_paths
        ]
        + [
            [img_2_arr(path, grayscale=True, resize=True, size=size), 1]
            for path in all_bact_img_paths
        ]
        + [
            [img_2_arr(path, grayscale=True, resize=True, size=size), 2]
            for path in all_viral_img_paths
        ]
    )
    return np.array(dataset, dtype="object")

# Data Processing

In [ ]:
# Retrieve data
# Load the dataset.pickle data. If not found, the image data will be manually
# processed again.
try:
    #with open('drive/MyDrive/HeartLabTestFiles/dataset.pickle', 'rb') as handle:
    with open('dataset.pickle', 'rb') as handle:
        dataset = pickle.load(handle)
except:
    dataset = create_datasets(data_dir)
    with open('dataset.pickle', 'wb') as handle:
        pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Split Data into training and validation sets

In [ ]:
# Split into training and testing sets
images = [data[0] for data in dataset]
labels = [data[1] for data in dataset]
images_train, images_test,labels_train, labels_test = train_test_split(images, labels, test_size=0.2)
images_train, images_test,labels_train, labels_test = np.array(images_train), np.array(images_test), np.array(labels_train), np.array(labels_test)
images = np.array(images).reshape(len(images), IMG_SIZE, IMG_SIZE, 1)
images_train = images_train.reshape(len(images_train), IMG_SIZE, IMG_SIZE, 1)
images_test = images_test.reshape(len(images_test), IMG_SIZE, IMG_SIZE, 1)
labels_train = tf.keras.utils.to_categorical(labels_train)
labels_test = tf.keras.utils.to_categorical(labels_test)
labels = tf.keras.utils.to_categorical(labels)

# Model Optimisation

Would you like to run the Hyperparameter Optimisation algorithm?  
This is not necessary, as I will demonstrate the optimised model at the end of the notebook

In [ ]:
run_hyper_param_opt = False #@param {type:"boolean"}

## Hyperparameter Optimisation

In [ ]:
# Iteratively create a new model
opt = ['agd', 'rmsprop', 'adam']
loss = ['categorical_crossentropy', 'poisson']
conv_layers = [1, 2, 3]
dense_layers = [1, 2, 3 , 4, 5]
layer_sizes = [32, 64, 128, 256]
dropout = [0, 0.2, 0.4, 0.8]
epochs = [10, 12, 15, 18, 20]

# Set this to false by default as you will probably not want to train and test over 100
# Convolutional Neural Networks
if run_hyper_param_opt:
  for dense_layer in dense_layers:
      for layer_size in layer_sizes:
          for drop in dropout:
              for epoch in epochs:
                name = "10_Epoch_Single_No_Dropout_{}_nodes_{}_dense_{}_time".format(drop, layer_size, dense_layer, int(time.time()))
                tensorboard = TensorBoard(log_dir='logs/{}'.format(name))
                print(name)
                model = Sequential()
                # Convolutional/Input Layer 
                model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(256,256,1)))
                model.add(MaxPool2D(pool_size=(1,1)))
                # flatten output of conv (input) layer 
                model.add(Flatten())

                # Variable number of Dense Layers
                for i in range(dense_layer):
                    model.add(Dense(layer_size, activation='relu'))  
                model.add(Dropout(drop))

                # Final layer
                model.add(Dense(layer_size, activation='relu'))

                # output layer
                model.add(Dense(3, activation='softmax'))

                # compiling the sequential model
                model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
                model.fit(images_train, labels_train, epochs=epoch, batch_size=20, validation_data=(images_test,labels_test), callbacks = [tensorboard])

# Analysing results with TensorBoard
Ensure that the `'log'` folder has been uploaded into the files workspace on the left in order to view the TensorBoard with the results.

In [ ]:
%load_ext tensorboard
!rm -rf ./logs/
%tensorboard --logdir drive/MyDrive/HeartLabTestFiles/logs
#%tensorboard --logdir logs

# The Selected Model
The following model has been selected for its optimal output from the TensorBoard analysis.

In [ ]:
name = "12_Epoch_Single_0.2_Dropout_256_nodes_3_dense_{}_time".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(name))
print(name)
model_opt = Sequential()
# Convolutional/Input Layer 
model_opt.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(256,256,1)))
model_opt.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv (input) layer 
model_opt.add(Flatten())

# Variable number of Dense Layers
model_opt.add(Dense(256, activation='relu'))
model_opt.add(Dense(256, activation='relu'))
model_opt.add(Dropout(0.2))

# Final layer
model_opt.add(Dense(256, activation='relu'))

# output layer
model_opt.add(Dense(3, activation='softmax'))

# compiling the sequential model
model_opt.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model_opt.fit(images_train, labels_train, epochs=15, batch_size=20, validation_data=(images_test,labels_test), callbacks = [tensorboard])

12_Epoch_Single_0.2_Dropout_256_nodes_3_dense_1629614446_time
Epoch 1/15
40/40 [==============================] - 43s 357ms/step - loss: 2548.9668 - accuracy: 0.4154 - val_loss: 167.7174 - val_accuracy: 0.5909
Epoch 2/15
40/40 [==============================] - 13s 323ms/step - loss: 79.0020 - accuracy: 0.6717 - val_loss: 16.2716 - val_accuracy: 0.8586
Epoch 3/15
40/40 [==============================] - 13s 319ms/step - loss: 16.5854 - accuracy: 0.8169 - val_loss: 10.5394 - val_accuracy: 0.8889
Epoch 4/15
40/40 [==============================] - 13s 318ms/step - loss: 4.8802 - accuracy: 0.9091 - val_loss: 8.7300 - val_accuracy: 0.8838
Epoch 5/15
40/40 [==============================] - 13s 320ms/step - loss: 2.0875 - accuracy: 0.9571 - val_loss: 10.4905 - val_accuracy: 0.8889
Epoch 6/15
40/40 [==============================] - 13s 324ms/step - loss: 2.5347 - accuracy: 0.9381 - val_loss: 8.1088 - val_accuracy: 0.8687
Epoch 7/15
40/40 [==============================] - 13s 323ms/step - l

# Testing the Selected model

In [ ]:
model_opt.evaluate(images_test, labels_test)
model_opt.evaluate(images, labels)
# Save the model
!mkdir -p saved_model
model_opt.save('saved_model/model_opt')

31/31 [==============================] - 2s 57ms/step - loss: 1.5830 - accuracy: 0.9818
INFO:tensorflow:Assets written to: saved_model/model_opt/assets
